# Setup (should take < 100 seconds)

In [1]:
import pandas as pd
shit = []
for i in range(1, 11):
    shit.append(pd.read_pickle('{}.pkl'.format(i)))

In [2]:
shit[0].keys()

dict_keys(['states', 'actions', 'values', 'masks', 'rewards', 'actions_probs', 'actions_onehot'])

In [ ]:
states_r = shit[iters//2]['states']
actions_r = shit[iters//2]['actions']
values_r = shit[iters//2]['values']
masks_r = shit[iters//2]['masks']
rewards_r = shit[iters//2]['rewards']
actions_probs_r = shit[iters//2]['actions_probs']
actions_onehot_r = shit[iters//2]['actions_onehot']

ACTIONS = OrderedDict({
'ACC': np.array([0, 1, 0]),
'IDLE': np.array([0, 0, 0]),
'BR': np.array([0, 0, 0.4]),
'LEFT': np.array([-1, 0, 0]),
'RIGHT': np.array([1, 0, 0]),
'LEFT_BR': np.array([-1, 0, 0.4]),
'RIGHT_BR': np.array([1, 0, 0.4]),
})

# Now, you can run it:

In [3]:
import gym
import numpy as np
from collections import OrderedDict
import tensorflow as tf
from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras.applications.mobilenet_v2 import MobileNetV2
import random

clipping_val = 0.2
critic_discount = 0.5
entropy_beta = 0.01
gamma = 0.99
lmbda = 0.95

ACTIONS = OrderedDict({
    'ACC': [0, 1, 0],
    'IDLE': [0, 0, 0],
    'BR': [0, 0, 1],
#     'LEFT': [-1, 0, 0],
#     'RIGHT': [1, 0, 0],
    'LEFT_ACC': [-1, 0.5, 0],
    'RIGHT_ACC': [1, 0.5, 0],
#     'LEFT_BR': [-1, 0, 0.5],
#     'RIGHT_BR': [1, 0, 0.5]
})

for i in range(100):
    ACTIONS['{}'.format(i)] = [random.uniform(-1,1), random.random(), random.random()/2] 

def get_advantages(values, masks, rewards):
    returns = []
    gae = 0
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * values[i + 1] * masks[i] - values[i]
        gae = delta + gamma * lmbda * masks[i] * gae
        returns.insert(0, gae + values[i])

    adv = np.array(returns) - values[:-1]
    std_adv = (adv - np.mean(adv)) / (np.std(adv) + 1e-10)
    return returns, std_adv


def ppo_loss_print(oldpolicy_probs, advantages, rewards, values):
    def loss(y_true, y_pred):
        y_true = tf.Print(y_true, [y_true], 'y_true: ')
        y_pred = tf.Print(y_pred, [y_pred], 'y_pred: ')
        newpolicy_probs = y_pred
        # newpolicy_probs = y_true * y_pred
        newpolicy_probs = tf.Print(newpolicy_probs, [newpolicy_probs], 'new policy probs: ')

        ratio = K.exp(K.log(newpolicy_probs + 1e-10) - K.log(oldpolicy_probs + 1e-10))
        ratio = tf.Print(ratio, [ratio], 'ratio: ')
        p1 = ratio * advantages
        p2 = K.clip(ratio, min_value=1 - clipping_val, max_value=1 + clipping_val) * advantages
        actor_loss = -K.mean(K.minimum(p1, p2))
        actor_loss = tf.Print(actor_loss, [actor_loss], 'actor_loss: ')
        critic_loss = K.mean(K.square(rewards - values))
        critic_loss = tf.Print(critic_loss, [critic_loss], 'critic_loss: ')
        term_a = critic_discount * critic_loss
        term_a = tf.Print(term_a, [term_a], 'term_a: ')
        term_b_2 = K.log(newpolicy_probs + 1e-10)
        term_b_2 = tf.Print(term_b_2, [term_b_2], 'term_b_2: ')
        term_b = entropy_beta * K.mean(-(newpolicy_probs * term_b_2))
        term_b = tf.Print(term_b, [term_b], 'term_b: ')
        total_loss = term_a + actor_loss - term_b
        total_loss = tf.Print(total_loss, [total_loss], 'total_loss: ')
        return total_loss

    return loss


def ppo_loss(oldpolicy_probs, advantages, rewards, values):
    def loss(y_true, y_pred):
        newpolicy_probs = y_pred
        ratio = K.exp(K.log(newpolicy_probs + 1e-10) - K.log(oldpolicy_probs + 1e-10))
        p1 = ratio * advantages
        p2 = K.clip(ratio, min_value=1 - clipping_val, max_value=1 + clipping_val) * advantages
        actor_loss = -K.mean(K.minimum(p1, p2))
        critic_loss = K.mean(K.square(rewards - values))
        total_loss = critic_discount * critic_loss + actor_loss - entropy_beta * K.mean(
            -(newpolicy_probs * K.log(newpolicy_probs + 1e-10)))
        return total_loss

    return loss


def get_model_actor_image(input_dims, output_dims):
    state_input = Input(shape=input_dims)
    oldpolicy_probs = Input(shape=(1, output_dims,))
    advantages = Input(shape=(1, 1,))
    rewards = Input(shape=(1, 1,))
    values = Input(shape=(1, 1,))

    feature_extractor = MobileNetV2(include_top=False, weights='imagenet')

    for layer in feature_extractor.layers:
        layer.trainable = False

    # Classification block
    x = Flatten(name='flatten')(feature_extractor(state_input))
    x = Dense(1024, activation='relu', name='fc1')(x)
    out_actions = Dense(n_actions, activation='softmax', name='predictions')(x)

    model = Model(inputs=[state_input, oldpolicy_probs, advantages, rewards, values],
                  outputs=[out_actions])
    model.compile(optimizer=Adam(lr=1e-3), loss=[ppo_loss(
        oldpolicy_probs=oldpolicy_probs,
        advantages=advantages,
        rewards=rewards,
        values=values)])
    model.summary()
    return model


def get_model_critic_image(input_dims):
    state_input = Input(shape=input_dims)

    feature_extractor = MobileNetV2(include_top=False, weights='imagenet')

    for layer in feature_extractor.layers:
        layer.trainable = False

    # Classification block
    x = Flatten(name='flatten')(feature_extractor(state_input))
    x = Dense(1024, activation='relu', name='fc1')(x)
    out_actions = Dense(1, activation='tanh')(x)

    model = Model(inputs=[state_input], outputs=[out_actions])
    model.compile(optimizer=Adam(lr=1e-3), loss='mse')
    model.summary()
    return model


# def test_reward():
#     state = env.reset()
#     done = False
#     total_reward = 0
#     print('testing...')
#     limit = 0
#     while not done:
#         state_input = K.expand_dims(state, 0)
#         action_probs = model_actor.predict([state_input, dummy_n, dummy_1, dummy_1, dummy_1], steps=1)
#         action = np.argmax(action_probs)
#         next_state, reward, done, _ = env.step(action)
#         state = next_state
#         total_reward += reward
#         limit += 1
#         if limit > 20:
#             break
#     return total_reward

def test_reward():
    state = env.reset()
    done = False
    total_reward = 0
    print('testing...')
    limit = 0
    while not done:
        state_input = K.expand_dims(state, 0)
        action_probs = model_actor.predict([state_input, dummy_n, dummy_1, dummy_1, dummy_1], steps=1)
        action_ind = np.argmax(action_probs)
        action = list(ACTIONS.values())[action_ind]
        next_state, reward, done, _ = env.step(action)
        env.render()
        state = next_state
        total_reward += reward
        limit += 1
        if limit > 200:
            break
    return total_reward

def one_hot_encoding(probs):
    one_hot = np.zeros_like(probs)
    one_hot[:, np.argmax(probs, axis=1)] = 1
    return one_hot


env = gym.make("CarRacing-v0")
state = env.reset()
state_dims = env.observation_space.shape
n_actions = len(ACTIONS)

dummy_n = np.random.rand(1, 1, n_actions)
dummy_1 = np.zeros((1, 1, 1))

tensor_board = TensorBoard(log_dir='./logs')

model_actor = get_model_actor_image(input_dims=state_dims, output_dims=n_actions)
model_critic = get_model_critic_image(input_dims=state_dims)

ppo_steps = 1024
target_reached = False
best_reward = 0
iters = 0
max_iters = 25
while not target_reached and iters < max_iters:

    states = []
    actions = []
    values = []
    masks = []
    rewards = []
    actions_probs = []
    actions_onehot = []
    state_input = None

    for itr in range(ppo_steps):
        state_input = K.expand_dims(state, 0)
        action_dist = model_actor.predict([state_input, dummy_n, dummy_1, dummy_1, dummy_1], steps=1)
        if iters == 0:
            action_dist = np.array([1/len(ACTIONS) for i in range(len(ACTIONS))]).reshape(1,len(ACTIONS))
        q_value = model_critic.predict([state_input], steps=1)
        action_ind = np.random.choice(n_actions, p=action_dist[0, :])
        action_onehot = np.zeros(n_actions)
        action_onehot[action_ind] = 1
        action = list(ACTIONS.values())[action_ind]

        observation, reward, done, info = env.step(action)
        env.render()
        print('itr: ' + str(itr) + ', action=' + str(action) + ', reward=' + str(reward) + ', q val=' + str(q_value))
        mask = not done

        states.append(state)
        actions.append(action)
        actions_onehot.append(action_onehot)
        values.append(q_value)
        masks.append(mask)
        rewards.append(reward)
        actions_probs.append(action_dist)

        state = observation
        if done:
            env.reset()

    q_value = model_critic.predict(state_input, steps=1)
    values.append(q_value)
    returns, advantages = get_advantages(values, masks, rewards)
    actor_loss = model_actor.fit(
        [states, actions_probs, advantages, np.reshape(rewards, newshape=(-1, 1, 1)), values[:-1]],
        [(np.reshape(actions_onehot, newshape=(-1, n_actions)))], verbose=True, shuffle=True, epochs=8,
        callbacks=None)
    critic_loss = model_critic.fit([states], [np.reshape(returns, newshape=(-1, 1))], shuffle=True, epochs=8,
                                   verbose=True, callbacks=None)

    avg_reward = np.mean([test_reward() for _ in range(1)])
    print('total test reward=' + str(avg_reward))
    if avg_reward > best_reward:
        print('best reward=' + str(avg_reward))
        model_actor.save('model_actor_{}_{}.hdf5'.format(iters, avg_reward))
        model_critic.save('model_critic_{}_{}.hdf5'.format(iters, avg_reward))
        best_reward = avg_reward
    if best_reward > 100 or iters > max_iters:
        target_reached = True
    iters += 1
    env.reset()

env.close()

Using TensorFlow backend.
D:\Anaconda\envs\CarRace\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1039..1305 -> 266-tiles track
retry to generate track (normal if there are not many instances of this message)
Track generation: 1235..1548 -> 313-tiles track


D:\Anaconda\envs\CarRace\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_224 (Model) multiple                  2257984   
_________________________________________________________________
flatten (Flatten)            (None, 11520)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              11797504  
_________________________________________________________________
predictions (Dense)          (None, 7)                 7175      
Total params: 14,062,663
Trainable params: 11,804,679
Non-trainable params: 2,257,984
_________________________________________________________________
Model: "model_2"
_________________________________________________________________
Layer (type)          

IndexError: list index out of range

In [ ]:
a = np.reshape(shit[iters//2]['rewards'], newshape=(-1, 1, 1))

In [ ]:
np.reshape(shit[iters//2]['actions_onehot'], newshape=(-1, n_actions)).shape

In [ ]:
values[:-1]